# Social Network Analysis

## 4. Equivalence Based Role Mining

Task: Grouping users into equivalence classes that define roles
Reference: Doran, D. (2017). Network Role Mining and Analysis. Springer International Publishing, Chapter 3.

In [ ]:
%pip install networkx[default]

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

## The Data

The data set is provided by Der Standard, one of the top Austrian newspapers.
In the online Standard people can post comments below articles and up/down vote comments.
The data set used in this handson and further in the project part of the course will consider a sample of those articles, comments, and votes. 

In [ ]:
df1 = pd.read_csv('data/Postings_01052019_15052019.csv', sep=';')
df2 = pd.read_csv('data/Postings_16052019_31052019.csv', sep=';')
df = pd.concat([df1, df2], ignore_index=True)
df.head()

There are different entities in the data set: 
* Users - identified by *ID_CommunityIdentity* (or *UserCommunityName*)
* Postings - identified by *ID_Posting*
* Articles - identified by *ID_Article*

Thus, there are different possibilities to build networks based on the posting data. 
We will concentrate now on the ***reply-to-network***. 


In [ ]:
votes = pd.read_csv('./data/Votes_01052019_15052019.csv', sep=';')
votes.head()

In [ ]:
votes['UserGender'].value_counts()